In [20]:
import os

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision.datasets
import torchvision.transforms as transforms
import torchvision

from mean_teacher import datasets, architectures

from IPython.display import clear_output

device = "cuda" if torch.cuda.is_available() else "cpu"

import matplotlib.pyplot as plt

to_image = transforms.ToPILImage()

In [2]:
dataset_config = datasets.__dict__['sslMini']()

In [3]:
def load_weights(model_arch, pretrained_model_path, state_dict, cuda=True):
        # Load pretrained model
        pretrained_model = torch.load(f=pretrained_model_path, map_location="cuda" if cuda else "cpu")

        from collections import OrderedDict
        new_state_dict = OrderedDict()
        for k, v in pretrained_model[state_dict].items():
            name = k[7:] # remove `module.`
            new_state_dict[name] = v

        # Load pre-trained weights in current model
        with torch.no_grad():
            model_arch.load_state_dict(new_state_dict, strict=True)

        # Debug loading
        #print('Parameters found in pretrained model:')
        pretrained_layers = new_state_dict.keys()
        #for l in pretrained_layers:
        #    print('\t' + l)
        #print('')

        for name, module in model_arch.state_dict().items():
            if name in pretrained_layers:
                assert torch.equal(new_state_dict[name].cpu(), module.cpu())
                #print('{} have been loaded correctly in current model.'.format(name))
            else:
                raise ValueError("state_dict() keys do not match")
                
        return model_arch


In [29]:
BATCH_SIZE = 64

evaldir = "/scratch/ijh216/ssl_mini/supervised/val"

eval_loader = torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(evaldir, dataset_config['eval_transformation']),
                                              batch_size=BATCH_SIZE,
                                              shuffle=False,
                                              num_workers=2,
                                              #pin_memory=True,
                                              drop_last=False)


In [39]:
traindir = "/scratch/ijh216/ssl_mini/supervised/train"

train_loader = torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(traindir, dataset_config['train_transformation']),
                                              batch_size=32,
                                              shuffle=False,
                                              num_workers=2,
                                              #pin_memory=True,
                                              drop_last=False)

In [7]:
model_dir = "/scratch/ijh216/ssl/ssl_shake_mini_augment/2019-05-06_23-27-41/10/transient/checkpoint.170.ckpt" 
model = architectures.__dict__['cifar_shakeshake26']().to(device)
model = load_weights(model, model_dir, state_dict="ema_state_dict", cuda=False)

In [74]:
target.ne(-1)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)

In [8]:
n_samples = 0.
n_correct_top_1 = 0
n_correct_top_k = 0

for i, (img, target) in enumerate(eval_loader):
    img, target = img.to(device), target.to(device)
    n_samples += BATCH_SIZE

        # Forward
    output = model(img)[0]

        # Top 1 accuracy
    pred_top_1 = torch.topk(output, k=1, dim=1)[1]
    n_correct_top_1 += pred_top_1.eq(target.view_as(pred_top_1)).int().sum().item()

        # Top k accuracy
    pred_top_k = torch.topk(output, k=5, dim=1)[1]
    target_top_k = target.view(-1, 1).expand(BATCH_SIZE, 5)
    n_correct_top_k += pred_top_k.eq(target_top_k).int().sum().item()
    
    if i % 10 == 0:
        clear_output()
        print(n_correct_top_1/n_samples)
        print(n_correct_top_k/n_samples)
    
    # Accuracy
top_1_acc = n_correct_top_1/n_samples
top_k_acc = n_correct_top_k/n_samples

print(top_1_acc)
print(top_k_acc)

0.046875
0.078125


KeyboardInterrupt: 